In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [6]:
import os
from langsmith_tracker import LangsmithTracker

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
tracker = LangsmithTracker(
    project_name=os.getenv("LANGCHAIN_PROJECT"), enable_tracking=True
)

# 인스턴스의 메서드를 호출합니다.
tracker.configure_tracking()

Langsmith 추적이 활성화되었습니다. [프로젝트명: langchain_note_2024]


### PromptTemplate

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 1. form_template() 를 사용한 prompt 생성 방법입니다.

- 치환될 변수를 `{ 변수 }` 로 묶어서 템플릿을 사용합니다.

In [7]:
from langchain_core.prompts import PromptTemplate

template = "{player} 는 어느 팀 소속의 선수인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['player'], input_types={}, partial_variables={}, template='{player} 는 어느 팀 소속의 선수인가요?')

`Player` 변수에 값을 넣어 프롬프트를 완성할 수 있습니다.

In [8]:
prompt = prompt.format(player="리오날 메시")
prompt

'리오날 메시 는 어느 팀 소속의 선수인가요?'

In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

template = """
{player} 는 어느 팀 소속의 선수인가요?
[FORMAT] 을 참고하여 답변해주세요.

# FORMAT
선수이름: 
종목: 
소속팀:
연봉정보:
"""

prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm
answer = chain.invoke({"player": "리오넬 메시"})
print(answer.content)

선수이름: 리오넬 메시  
종목: 축구  
소속팀: 인터 마이애미 CF  
연봉정보: 약 4,000만 달러 (2023년 기준)  


### 2. PromptTemplate 객체 생성과 동시에 prompt를 생성하는 방법입니다.

- input_variables 을 지정합니다.

In [14]:
template = """
{player} 는 어느 팀 소속의 선수인가요?
[FORMAT] 을 참고하여 답변해주세요.

# FORMAT
선수이름: 
종목: 
소속팀:
연봉정보:
"""

prompt = PromptTemplate(template=template, input_variables=["player"])

chain = prompt | llm
answer = chain.invoke({"player": "리오넬 메시"})
print(answer.content)

선수이름: 리오넬 메시  
종목: 축구  
소속팀: 인터 마이애미 CF  
연봉정보: 약 4천만 달러 (2023년 기준)  


`partial_variables` : 처음에는 변동이 되는 변수이거나 변동이 예상되는 변수였지만, 확정되었을때 사용할 수 있는 문법입니다.

In [17]:
template = "{player} 의 {question1} 과(와) {question2} 정보를 알려주세요."

prompt = PromptTemplate(
    template=template,
    input_variables=["player", "question1"],
    partial_variables={"question2": "수상경력"},
)

prompt

PromptTemplate(input_variables=['player', 'question1'], input_types={}, partial_variables={'question2': '수상경력'}, template='{player} 의 {question1} 과(와) {question2} 정보를 알려주세요.')

In [18]:
prompt.format(player="리오넬 메시", question1="소속팀")

'리오넬 메시 의 소속팀 과(와) 수상경력 정보를 알려주세요.'

`from_template` 로 `patial` 을 사용하는 방법입니다.

In [28]:
prompt = PromptTemplate.from_template(template)
prompt_partial = prompt.partial(question2="수상경력")
prompt_partial

PromptTemplate(input_variables=['player', 'question1'], input_types={}, partial_variables={'question2': '수상경력'}, template='{player} 의 {question1} 과(와) {question2} 정보를 알려주세요.')

In [29]:
prompt_partial.format(player="리오넬 메시", question1="소속팀")

'리오넬 메시 의 소속팀 과(와) 수상경력 정보를 알려주세요.'

In [30]:
chain = prompt_partial | llm
answer = chain.invoke({"player": "리오넬 메시", "question1": "소속팀"})
print(answer.content)

리오넬 메시(Lionel Messi)는 2021년부터 파리 생제르맹(Paris Saint-Germain, PSG) 소속으로 활동하고 있습니다. 이전에는 FC 바르셀로나에서 오랜 기간 동안 활약했습니다.

메시의 주요 수상 경력은 다음과 같습니다:

### 개인 수상
- **발롱도르**: 7회 수상 (2009, 2010, 2011, 2012, 2015, 2019, 2021)
- **FIFA 올해의 선수**: 1회 수상 (2019)
- **UEFA 올해의 선수**: 1회 수상 (2011)
- **Golden Shoe**: 6회 수상 (2010, 2012, 2013, 2017, 2018, 2019)

### 클럽 수상 (FC 바르셀로나)
- **라리가**: 10회 우승
- **코파 델 레이**: 7회 우승
- **UEFA 챔피언스리그**: 4회 우승
- **슈퍼코파 데 에스파냐**: 8회 우승
- **FIFA 클럽 월드컵**: 3회 우승
- **UEFA 슈퍼컵**: 3회 우승

### 국가대표 수상 (아르헨티나)
- **코파 아메리카**: 1회 우승 (2021)
- **FIFA 월드컵**: 1회 우승 (2022)
- **FIFA U-20 월드컵**: 1회 우승 (2005)
- **올림픽 금메달**: 1회 수상 (2008)

메시는 축구 역사상 가장 위대한 선수 중 한 명으로 평가받고 있으며, 그의 경력은 계속해서 기록을 갱신하고 있습니다.


### `partial_variables` : 부분변수 채움

- partial 을 자주 사용하는 용도는 함수 호출할 때입니다. 항상 동일한 방식으로 가져오고 싶은 변수가 있는 경우에 자주 사용합니다.
- 대표적으로 날짜와 시간입니다. 또는 포맷이 있는 경우 사용합니다.

In [32]:
from datetime import datetime


def get_today():
    return datetime.now().strftime("%B %d")


get_today()

'November 11'

In [58]:
prompt = PromptTemplate(
    template="""
    모르거나 없는 정보라면 '정보가 없습니다.' 라고 답변해주세요. 유명인 정보를 나열하지 말아주세요.
    {today} 와 같은 날짜에 생일인 한국의 유명인 {n} 명을 나열해주세요.
    
    # FORMAT
    이름(생일)
    """,
    input_variables=["n"],
    partial_variables={"today": get_today},
)

chain = prompt | llm
answer = chain.invoke({"n": 3})
# answer = chain.invoke({"today":"Jan 02", "n": 3}) # today를 작성해주면 partial이 설정되어 있어도 내용을 덮어 쓰게 됩니다.
print(answer.content)

정보가 없습니다.
